# dSentra
pool number = 5
operator number = 6

In [101]:
# IGNORE THIS :3
import os
from web3.middleware import construct_sign_and_send_raw_middleware
from geode import Geode,globals
G = Geode(exec_api = os.environ['EXECUTION_API'], cons_key = os.environ['CONSENSUS_KEY'])
Portal = G.Portal
oid = Portal.functions.allIdsByType(globals.ID_TYPE.OPERATOR, 6).call()
pid = Portal.functions.allIdsByType(globals.ID_TYPE.POOL, 5).call()
myPool = Portal.pool(pid)
myOperator = Portal.operator(oid)
private_key = os.environ["PRIVATE_KEY"] # name it to something else!!!
acct = G.w3.eth.account.from_key(private_key)
G.w3.middleware_onion.add(construct_sign_and_send_raw_middleware(acct))
G.w3.eth.defaultAccount = acct
# IGNORE THIS :3

# Creating Validators

Validators are created in 2 steps: `proposeStake()` and `stake()`

1. proposeStake() will send 1 eth to beacon chain
2. stake() will send 31 eth to beacon chain

Take a look at this: https://docs.geode.fi/operator-marketplace/a-validators-lifecycle

Any Staking Pool can work with any Operator. It is up to them choosing their own subset. This is done by the `delegate` function, and is observed with the `allowance` function.

So, we have modified the staking-cli to give you the power to do 1 Eth and 31 Eth deposits, or any amount really...

https://github.com/Geodefi/staking-deposit-cli

### Get Prepared

0. Keep your `mnemonic` in mind
You can use a new mnemonic for every single pool, or you can use a one global mnemonic and increase its index over time. 
But, it is very important that you keep some consistency on your mnemonic usage, while keeping it safe.

I will use one global mnemonic. 

1. Detect which pools you can stake for
You can basically loop through all pools periodically and check the `allowance` function, or you can listen for `Delegation` event. It is totally up to you how you handle this.
However, keep in mind, other node operators racing with you to grab delegations every time pool `surplus` hits to 32 eth.

This is particularly important because proposeStake function can be called to propose multiple validators for one pool.
stake() function, on the other hand can be called with multiple validator pubkeys from mulitple pools.
These function calls will be kind of expensive. So, be wise spending your gas :)

> Pro tip: order your pubkeys according to their pools to save gas. For example use `[pool1, pool1, pool1, pool2, pool2, pool3]` instead of 
`[pool1, pool2, pool3, pool1, pool2, pool1].`


> Delegation Event ABI:
> 
> ```
>     {
>       "anonymous": false,
>       "inputs": [
>         {
>           "indexed": false,
>           "internalType": "uint256",
>           "name": "poolId",
>           "type": "uint256"
>         },
>         {
>           "indexed": true,
>           "internalType": "uint256",
>           "name": "operatorId",
>           "type": "uint256"
>         },
>         {
>           "indexed": false,
>           "internalType": "uint256",
>           "name": "allowance",
>           "type": "uint256"
>         }
>       ],
>       "name": "Delegation",
>       "type": "event"
>     }
> ```

2. Create 2 deposit-data for every validator for every validator proposal. One of the most important thing here is to use pool's `withdrawal_contract` as `eth1_withdrawal_address`
This can be done easily by:

```shell
python3 ./staking_deposit/deposit.py existing-mnemonic --num_validators = {{number_of_proposal}} --amount = 1 --chain=goerli --eth1_withdrawal_address {{withdrawal_contract}}

python3 ./staking_deposit/deposit.py existing-mnemonic --num_validators = {{number_of_proposal}} --amount = 31 --chain=goerli --eth1_withdrawal_address {{withdrawal_contract}}
```

3. validate the deposit-data

In [7]:
from geode.utils.bls import validate_deposit_data_file
from geode.globals import DEPOSIT_SIZE, Network

# lets first validate
##  use https://github.com/Geodefi/staking-deposit-cli to generate
propose_data_path="deposit_data/deposit_data-1692015960.json"
stake_data_path="deposit_data/deposit_data-1692016808.json"

# doesn't look for deposit_message_root and deposit_data_root issues as they are not used.
validate_deposit_data_file(
    deposit_data_path=stake_data_path,
    amount=DEPOSIT_SIZE.STAKE,
    credential=myPool.withdrawalCredential[2:], # remove the 0x part 
    network= Network.goerli
    )

validate_deposit_data_file(
    deposit_data_path=propose_data_path,
    amount=DEPOSIT_SIZE.PROPOSAL, 
    credential=myPool.withdrawalCredential[2:], # can also use Validator.withdrawalCredentials * if already actively validating validator *
    network= Network.goerli
)

[{'pubkey': '973ebcb9605afdcda77fac8c50f1ddbbb9f2bfbabbe41d3133a2adcb7d0c7c0a7fa1bc6736dbdb2784cc70ca86b0b2b1',
  'withdrawal_credentials': '010000000000000000000000914a8af2c1c88b4b63afaa8964f2d66d533ec6c8',
  'amount': 1000000000,
  'signature': 'a913b6243846170fb2d406df00555128b97a70951fd063cacec455213f8a539ce774cc52a98f3c81ec20f454bca8bcb20075c9f7266b1161534b7ed60c4e6587d410c2fe4ff316a66574d719c3634deb854c082f72a32fe6969f51eb6faa233e',
  'deposit_message_root': '9bc178bac027002dc222d7faeecfdaed127e8f95dbbbd1588a5667cea4a3fd9e',
  'deposit_data_root': '3cf4d16f5f7bfa28b7e2984433cc83d57cde073badec0bf0b0675bd63990fabf',
  'fork_version': '00001020',
  'network_name': 'goerli',
  'deposit_cli_version': '2.4.0'}]

4. prepare the data using native SDK functions 
   
> __pub1 == pub31__. Since they are actually used to deposit into the same validator. So, be careful about the `index` prompt while using existing-mnemonic.

In [8]:
pubkeys1, sig1 = myPool.prepareProposeStake(deposit_data_path=propose_data_path)
pubkeys31, sig31 = myPool.prepareStake(deposit_data_path=stake_data_path)

In [9]:
#dSentra-branch 
print("Operator ID:",myOperator.ID, "Operator maintainer:",myOperator.maintainer, "Operator controller:", myOperator.CONTROLLER)
print("Pool ID:",myPool.ID, "Pool maintainer:",myPool.maintainer, "Pool CONTROLLER:",myPool.CONTROLLER)
print("PRIVATE ACCOUNT address:",acct.address)

Operator ID: 31714947982858204937006775680871004176266839432931470523383748182266983527862 Operator maintainer: 0x77DD4eF5Be22a3A6b4E8F26ff085B6C0C27673CD Operator controller: 0xBeAe83D58B6e26Ac6b906c4129e0D96722f5dEAa
Pool ID: 86502518581938563901485985101800597243059589530815947830543203898014478629305 Pool maintainer: 0x77DD4eF5Be22a3A6b4E8F26ff085B6C0C27673CD Pool CONTROLLER: 0xBeAe83D58B6e26Ac6b906c4129e0D96722f5dEAa
PRIVATE ACCOUNT address: 0x77DD4eF5Be22a3A6b4E8F26ff085B6C0C27673CD


5. Propose!

In [6]:
Portal.functions.proposeStake(myPool.ID, myOperator.ID, pubkeys1, sig1, sig31,).transact({"from": acct.address}) 

ContractLogicError: execution reverted: SML: used or alienated pk

In [114]:
pubkey="0x973ebcb9605afdcda77fac8c50f1ddbbb9f2bfbabbe41d3133a2adcb7d0c7c0a7fa1bc6736dbdb2784cc70ca86b0b2b1"
          
Portal.functions.canStake(pubkey).call()


True

6. Wait until approved...
Honestly, it should take a day max. But, you can listen `VerificationIndexUpdated` event, which is emitted every time Oracle approves a batch of proposals. Or, simply check the `Portal.canStake(pubkey)` function periodically.

> VerificationIndexUpdated Event ABI:
> 
> ```
>  {
>    "anonymous": false,
>    "inputs": [
>      {
>        "indexed": false,
>        "internalType": "uint256",
>        "name": "validatorVerificationIndex",
>        "type": "uint256"
>      }
>    ],
>    "name": "VerificationIndexUpdated",
>    "type": "event"
>  }
>```

In [113]:
#GEODE Portal contract instance VerificationIndexUpdated Event ABI:=
import json
abi = json.loads('[{"anonymous": false, "inputs": [{"indexed": false,"internalType": "uint256","name": "validatorVerificationIndex","type": "uint256"}],"name":"VerificationIndexUpdated","type": "event"},{"name":"ControllerChanged","type":"event", "inputs":[]}]')
#GEODE Portal contract  - proxy instance - Address
contract_address = "0xFAF3b3eE96Fa210Cf196CDa9015384dE049Fa2a8"
contract_instance = G.w3.eth.contract(address= contract_address, abi=abi)
# fetch IndexUpdated events in the last block
params = {"fromBlock": G.w3.eth.block_number}
logs = contract_instance.events.VerificationIndexUpdated.get_logs(params)
print(logs)


()


7. Stake!

In [115]:
Portal.functions.stake(myOperator.ID, pubkeys1).transact({"from": acct.address})

HexBytes('0x0426461d0adfa118cfd700e0b38a09da495e4fbb2b5156f62ec1c054e778e227')